# Master Prepare Notebook

Deze notebook wordt gebruikt om alle data uit de datasets in te laden en verder te verwerken, zodat deze klaar staat om modellen te trainen.

In [1]:
# Load public modules.
import os, sys
from sklearn.pipeline import Pipeline
from pathlib import Path
import pickle
import time

# Get the home dir and username.
HOME = Path.home()
USERNAME = os.path.basename(HOME)

# Set codebase path for old VAO.
CODEBASE_PATH_OLD = os.path.join(HOME, 'Documents/woonfraude/codebase/')
sys.path.insert(1, CODEBASE_PATH_OLD)
                
# Set codebase path for new VAO.
CODEBASE_PATH_NEW = os.path.join('/data', USERNAME, 'Documents/woonfraude/codebase/')
sys.path.insert(1, CODEBASE_PATH_NEW)

# Import own modules.
from datasets import *
from clean import *
from extract_features import *

In [2]:
# Set global variables.
FORCE_DOWNLOAD = False
FORCE_DATASET_SPECIFIC_PREPROCESSING = False

# Load all datasets in memory

In [3]:
##############################
# Initialize dataset objects #
##############################

adresDataset = AdresDataset()
zakenDataset = ZakenDataset()
stadiaDataset = StadiaDataset()
personenDataset = PersonenDataset()
bagDataset = BagDataset()
hotlineDataset = HotlineDataset()

In [4]:
#######################################################################################
# Download data and perform dataset-specific pre-processing steps for dataset objects #
#######################################################################################

# Forces the downloading of new data.
if FORCE_DOWNLOAD:
    adresDataset.download(force=True)
    zakenDataset.download(force=True)
    stadiaDataset.download(force=True)
    personenDataset.download(force=True)
    bagDataset.download(force=True)
    

 # Forces the dataset specific pre-processing of the downloaded data.
if FORCE_DATASET_SPECIFIC_PREPROCESSING:
    
    # Adres dataset.
    adresDataset.load('download')
    adresDataset.extract_leegstand()
    adresDataset.enrich_with_woning_id()

    # Zaken Dataset.
    zakenDataset.load('download')
    zakenDataset.add_categories()
    zakenDataset.filter_categories()  # Verwijder meldingen met categorieeen "woningkwaliteit" en "afdeling vergunningen en beheer".

    # Stadia dataset.
    stadiaDataset.load('download')
    stadiaDataset.add_zaak_stadium_ids()
    stadiaDataset.add_labels()

    # Bag dataset.
    bagDataset.download(force=True)
    bagDataset.load('download')
    bagDataset.bag_fix()

    # Hotline dataset.
    hotlineDataset.download(force=True)

In [5]:
###################
## Load datasets ##
###################
# Load datasets from cache (when download and pre-processing steps in previous block have been done).

adresDataset.load('download_leegstand_woningId')
zakenDataset.load('download_categories_filterCategories')
stadiaDataset.load('download_ids_labels')
personenDataset.load('download')
bagDataset.load('download_columnFix')
hotlineDataset.load('download')

Version 'download_leegstand_woningId' of dataset 'adres' loaded!
Version 'download_categories_filterCategories' of dataset 'zaken' loaded!
Version 'download_ids_labels' of dataset 'stadia' loaded!
Version 'download' of dataset 'personen' loaded!
Version 'download_columnFix' of dataset 'bag' loaded!
Version 'download' of dataset 'hotline' loaded!


# Clean and extract features from all datasets

In [6]:
#########################
## Clean zaken dataset ##
#########################

zakenPipeline = Pipeline(steps=[
    ('clean', CleanTransformer(
        id_column=zakenDataset.id_column,
        drop_duplicates=True,
        fix_date_columns=['begindatum','einddatum', 'wzs_update_datumtijd'],
        clean_dates=True,
        lower_string_columns=True,
        impute_missing_values=True,
        impute_missing_values_custom={'categorie': 'missing'})
    )])

zaken = zakenPipeline.fit_transform(zakenDataset.data)

Dataframe "zaken": Dropped 0 duplicates!
Dataframe "zaken": Fixed dates!
Dataframe "zaken": Cleaned out 3 dates!
Lowered strings of cols ['beh_code', 'beh_oms', 'afg_code_beh', 'afs_code', 'afs_oms', 'afg_code_afs', 'eigenaar', 'zaak_id', 'mededelingen', 'categorie'] in df zaken!
Missing values in df zaken have been imputed!
Missing values (using custom strategy) of cols ['categorie'] in df zaken have been imputed!


In [7]:
##########################
## Clean stadia dataset ##
##########################

stadiaPipeline = Pipeline(steps=[
    ('clean', CleanTransformer(
        id_column=stadiaDataset.id_column,
        drop_duplicates=True,
        fix_date_columns=['begindatum', 'peildatum', 'einddatum', 'date_created',
                          'date_modified', 'wzs_update_datumtijd'],
        clean_dates=True,
        lower_string_columns=True,
        impute_missing_values=True)
    )])

stadia = stadiaPipeline.fit_transform(stadiaDataset.data)

Dataframe "stadia": Dropped 0 duplicates!
Dataframe "stadia": Fixed dates!
Dataframe "stadia": Cleaned out 0 dates!
Lowered strings of cols ['afg_co', 'sta_code', 'sta_oms', 'afg_code_stad', 'afs_code', 'afs_oms', 'afg_code_afs', 'resultaat', 'mdr_code', 'user_created', 'user_modified', 'stadia_id', 'zaak_id', 'stadium_id', 'label'] in df stadia!
Missing values in df stadia have been imputed!


In [8]:
############################
## Clean personen dataset ##
############################

personenPipeline = Pipeline(steps=[
    ('clean', CleanTransformer(
        id_column=personenDataset.id_column,
        drop_duplicates=True,
        fix_date_columns=['geboortedatum'],
        lower_string_columns=True)
    )])

personen = personenPipeline.fit_transform(personenDataset.data)

Dataframe "personen": Dropped 0 duplicates!
Dataframe "personen": Fixed dates!
Lowered strings of cols ['pen_type', 'gezinsverhouding', 'geslacht', 'voorletters', 'a_nr_gezinshoofd', 'burgerlijke_staat', 'naam', 'geheim_adres', 'voorv_mnaam', 'voorv_naam', 'meisjesnaam', 'vertrekdatum_adam', 'ind_naamgebruik', 'nat_ned', 'ind_nat_ovlp', 'verblijfstatus', 'datum_einde_vblstat', 'landcode', 'user_created', 'user_modified', 'datum_begin_vblstat', 'ais_nr', 'crv_nr', 'geheim', 'anr_ouder_1', 'anr_ouder_2', 'anr_partner', 'in_onderzoek', 'datum_verkrijging_vreemd', 'voorletters_zdia', 'naam_zdia', 'voorv_mnaam_zdia', 'voorv_naam_zdia', 'meisjesnaam_zdia', 'nm_dia_255', 'mnm_dia_255'] in df personen!
Missing values in df personen have been imputed!


In [9]:
#######################
## Clean BAG dataset ##
#######################

bagPipeline = Pipeline(steps=[
    ('clean', CleanTransformer(
        id_column=bagDataset.id_column,
        drop_duplicates=True,
        fix_date_columns=[],
        drop_columns = ['indicatie_geconstateerd', 'indicatie_in_onderzoek', 'woningvoorraad'],
        lower_string_columns=True,
        impute_missing_values=True,
        impute_missing_values_mode=['status_coordinaat_code'],
        fillna_columns={'_huisnummer_verblijfsobject': 0,
                         '_huisletter_verblijfsobject': 'None',
                         '_openbare_ruimte_naam_verblijfsobject': 'None',
                         '_huisnummer_toevoeging_verblijfsobject': 'None',
                         'type_woonobject_omschrijving': 'None',
                         'eigendomsverhouding_id': 'None',
                         'financieringswijze_id': -1,
                         'gebruik_id': -1,
                         'reden_opvoer_id': -1,
                         'status_id_verblijfsobject': -1,
                         'toegang_id': 'None'})
    )])

bagDataset.data = bagPipeline.fit_transform(bagDataset.data)
bag = bagDataset.data  # For easier usage.

Dataframe "bag": Dropped 0 duplicates!
Lowered strings of cols ['id_nummeraanduiding', 'landelijk_id_nummeraanduiding', 'huisletter_nummeraanduiding', 'huisnummer_toevoeging_nummeraanduiding', 'postcode', 'type', 'adres_nummer', 'vervallen_nummeraanduiding', 'hoofdadres', '_openbare_ruimte_naam_nummeraanduiding', 'bron_id_nummeraanduiding', 'ligplaats_id', 'openbare_ruimte_id', 'standplaats_id', 'status_id_nummeraanduiding', 'verblijfsobject_id', '_geom', 'id_ligplaats', 'landelijk_id_ligplaats', 'vervallen_ligplaats', 'geometrie_ligplaats', 'bron_id_ligplaats', 'status_id_ligplaats', 'id_standplaats', 'landelijk_id_standplaats', 'vervallen_standplaats', 'geometrie_standplaats', 'bron_id_standplaats', 'status_id_standplaats', 'id_verblijfsobject', 'landelijk_id_verblijfsobject', 'status_coordinaat_code', 'status_coordinaat_omschrijving', 'type_woonobject_code', 'type_woonobject_omschrijving', 'geometrie_verblijfsobject', '_huisletter_verblijfsobject', 'bron_id_verblijfsobject', 'eigend

In [10]:
###########################
## Clean hotline dataset ##
###########################

hotlinePipeline = Pipeline(steps=[
    ('clean', CleanTransformer(
        id_column=hotlineDataset.id_column,
        drop_duplicates=True,
        lower_string_columns=True,
        impute_missing_values=True)
    )])

hotlineDataset.data = hotlinePipeline.fit_transform(hotlineDataset.data)
hotline = hotlineDataset.data  # For easier usage.

Dataframe "hotline": Dropped 0 duplicates!
Lowered strings of cols ['mdw_code', 'overtreding_code', 'melder_anoniem', 'melder_naam', 'melder_emailadres', 'melder_telnr', 'situatie_schets', 'user_created', 'user_modified'] in df hotline!
Missing values in df hotline have been imputed!


In [11]:
#########################
## Clean adres dataset ##
#########################

# Hier de extract stap weghalen? Deze past waarschijnlijk beter na het combinen v/d datasets.
adresPipeline = Pipeline(steps=[
    ('clean', CleanTransformer(
        id_column=adresDataset.id_column,
        drop_duplicates=True,
        fix_date_columns=['hvv_dag_tek', 'max_vestig_dtm', 'wzs_update_datumtijd'],
        lower_string_columns=True,
        impute_missing_values=True,
        fillna_columns={'hsnr': 0, 'sttnaam': 'None', 'hsltr': 'None', 'toev': 'None'})
    )])

adresDataset.data = adresPipeline.fit_transform(adresDataset.data)
adres = adresDataset.data  # For easier usage.

Dataframe "adres": Dropped 0 duplicates!
Dataframe "adres": Fixed dates!
Lowered strings of cols ['postcode', 'sdl_code', 'brt_code', 'pvh_cd', 'pvh_omschr', 'sbw_omschr', 'sbv_omschr', 'wzs_buurtcode_os_2015', 'wzs_buurtnaam_os_2015', 'wzs_buurtcombinatiecode_os_2015', 'wzs_buurtcombinatienaam_os_2015', 'wzs_22gebiedencode_os_2015', 'wzs_22gebiedennaam_os_2015', 'wzs_rayoncode_os_2015', 'wzs_rayonnaam_os_2015', 'wzs_stadsdeelcode_os_2015', 'wzs_stadsdeelnaam_os_2015', 'wzs_alternatieve_buurtennaam_os_2015', 'wzs_alternatieve_buurtencode_os_2015', 'wzs_geom', 'wzs_wijze_verrijking_geo', 'wzs_22gebiedencode_2015', 'wzs_22gebiedennaam_2015', 'sttnaam', 'hsltr', 'toev', 'brtcombi_naam', 'sdl_naam', 'brt_naam', 'a_dam_bag', 'landelijk_bag', 'hvv_dag_tek', 'max_vestig_dtm'] in df adres!
Missing values in df adres have been imputed!


# Combine the datasets

In [12]:
##########################
## Enrich adres dataset ##
##########################

# Enrich the adres dataset with information from the bag, personen and hotline datasets.
adresDataset.enrich_with_bag(bagDataset.data)
adresDataset.enrich_with_personen_features(personenDataset.data)
adresDataset.add_hotline_features(hotlineDataset.data)


##########################
## Enrich zaken dataset ##
##########################

# Only keep the finished cases in the zaken dataset (remove all unfinished cases).
zakenDataset.keep_finished_cases(stadiaDataset.data)

# Add a label to indicate woonfraude.
zakenDataset.add_binary_label_zaken(stadiaDataset.data)

Missing values in df adres have been imputed!
Saving version 'download_leegstand_woningId_bag' of dataframe 'adres'.


/usr/local/lib/python3.6/site-packages/pandas/core/generic.py:2377: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block1_values] [items->['postcode', 'sdl_code', 'brt_code', 'pvh_cd', 'pvh_omschr', 'sbw_omschr', 'sbv_omschr', 'wzs_buurtcode_os_2015', 'wzs_buurtnaam_os_2015', 'wzs_buurtcombinatiecode_os_2015', 'wzs_buurtcombinatienaam_os_2015', 'wzs_22gebiedencode_os_2015', 'wzs_22gebiedennaam_os_2015', 'wzs_rayoncode_os_2015', 'wzs_rayonnaam_os_2015', 'wzs_stadsdeelcode_os_2015', 'wzs_stadsdeelnaam_os_2015', 'wzs_alternatieve_buurtennaam_os_2015', 'wzs_alternatieve_buurtencode_os_2015', 'wzs_geom', 'wzs_wijze_verrijking_geo', 'wzs_22gebiedencode_2015', 'wzs_22gebiedennaam_2015', 'sttnaam', 'hsltr', 'toev', 'brtcombi_naam', 'sdl_naam', 'brt_naam', 'a_dam_bag', 'landelijk_bag', 'postcode_x', 'id_nummeraanduiding', 'landelijk_id_nummeraanduiding', 'huisletter_nummeraanduiding'

The adres dataset is now enriched with BAG data.
Now looping over all address ids that have a link with one or more inhabitants...
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
Now looping over all rows in the adres dataframe in order to add person information...
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
...done!
Saving version 'download_leegstand_woningId_bag_personen' of dataframe 'adres'.


/usr/local/lib/python3.6/site-packages/pandas/core/generic.py:2377: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block5_values] [items->['postcode', 'sdl_code', 'brt_code', 'pvh_cd', 'pvh_omschr', 'sbw_omschr', 'sbv_omschr', 'wzs_buurtcode_os_2015', 'wzs_buurtnaam_os_2015', 'wzs_buurtcombinatiecode_os_2015', 'wzs_buurtcombinatienaam_os_2015', 'wzs_22gebiedencode_os_2015', 'wzs_22gebiedennaam_os_2015', 'wzs_rayoncode_os_2015', 'wzs_rayonnaam_os_2015', 'wzs_stadsdeelcode_os_2015', 'wzs_stadsdeelnaam_os_2015', 'wzs_alternatieve_buurtennaam_os_2015', 'wzs_alternatieve_buurtencode_os_2015', 'wzs_geom', 'wzs_wijze_verrijking_geo', 'wzs_22gebiedencode_2015', 'wzs_22gebiedennaam_2015', 'sttnaam', 'hsltr', 'toev', 'brtcombi_naam', 'sdl_naam', 'brt_naam', 'a_dam_bag', 'landelijk_bag', 'postcode_x', 'id_nummeraanduiding', 'landelijk_id_nummeraanduiding', 'huisletter_nummeraanduiding'

The adres dataset is now enriched with personen data.
Saving version 'download_leegstand_woningId_bag_personen_hotline' of dataframe 'adres'.
The adres dataset is now enriched with hotline data.


/usr/local/lib/python3.6/site-packages/pandas/core/computation/expressions.py:183: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


Selected 18047 finished cases from a total of 33718 cases.
Saving version 'download_categories_filterCategories_finishedCases' of dataframe 'zaken'.


/usr/local/lib/python3.6/site-packages/pandas/core/generic.py:2377: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['afg_code_afs', 'afg_code_beh', 'afs_code', 'afs_oms', 'beh_code', 'beh_oms', 'categorie', 'eigenaar', 'mededelingen', 'zaak_id']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [13]:
###############################################################################
## Remove implicit label columns and superfluous columns  from adres dataset ##
###############################################################################

adres_remove = [# Remove because cols do not exists when melding is received
                    'wzs_update_datumtijd',
                    # Remove because cols do not add extra information.
                    'kmrs',
                    'straatcode',
                    'xref',
                    'yref',
                    'postcode',
                    'wzs_buurtcode_os_2015',
                    'wzs_buurtcombinatiecode_os_2015',
                    'wzs_stadsdeelcode_os_2015',
                    'hvv_dag_tek', # Empty column
                    'max_vestig_dtm', # Empty column
                    'wzs_22gebiedencode_os_2015', # Empty column
                    'wzs_22gebiedennaam_os_2015', # Empty column
                    'pvh_cd',
                    'sbv_code',
                    'sbw_code',
                    'wzs_wijze_verrijking_geo',
                    'wzs_22gebiedencode_2015',
                    'brt_naam',
                    'wzs_buurtnaam_os_2015',
                    'wzs_buurtcombinatienaam_os_2015',
                    'wzs_rayonnaam_os_2015',
                    'wzs_rayoncode_os_2015',
                    'wzs_stadsdeelnaam_os_2015',
                    'wzs_alternatieve_buurtennaam_os_2015',
                    'wzs_alternatieve_buurtencode_os_2015',
                    'wzs_geom',
                    'brt_code',
                    'brtcombi_code',
                    'brtcombi_naam',
                    'sdl_code',
                    'wzs_22gebiedennaam_2015',
                    'wzs_id',
                    'a_dam_bag',
                    'landelijk_bag']

bag_remove = ['einde_geldigheid',               # Only 2 entries in column.
              'verhuurbare_eenheden',           # Only ~2k entries in column.
              'geometrie_ligplaats',            # Needs a lot of processing before being useful.
              'bron_id_verblijfsobject',        # Only 2 entries in column.
              'locatie_ingang_id',              # Only 2 entries in column.
              'reden_afvoer_id',                # Only a few entries in column.
              '_gebiedsgerichtwerken_id',       # Superfluous (gebied).
              '_grootstedelijkgebied_id',       # Superfluous (grootstedelijkgebied).
              'buurt_id',                       # Superfluous (buurt).
              # ONDERSTAANDE 4 KOLOMMEN KONDEN EERDER NIET WEG IVM MATCH MET ADRES DATAFRAME.
              # DEZE MOETEN NU WEL WEG, DAAROM WORDT NU HIER ALLES WEGGEHAALD.
              '_openbare_ruimte_naam_nummeraanduiding',          # Superfluous (straatnaam).
              'vervallen_nummeraanduiding',
              'vervallen_ligplaats',
              'vervallen_standplaats',
              'vervallen_verblijfsobject',
              'document_mutatie',               # Not available at time of signal.
              'date_modified_nummeraanduiding', # Not available at time of signal.
              'document_nummer',                # Not needed? (Swaan?)
              'status_coordinaat_omschrijving', # Not needed? (Swaan?)
              'type_woonobject_code',           # Not needed? (Swaan?)
              'id_ligplaats',                   # Not needed.
              'landelijk_id_ligplaats',         # Not needed.
              'id_standplaats',                 # Not needed.
              'landelijk_id_standplaats',       # Not needed.
              'id_verblijfsobject',             # Not needed.
              'landelijk_id_verblijfsobject',   # Not needed.
              ]

# Remove the columns that are defined above from the dataset.
adresDataset.data.drop(columns=adres_remove + bag_remove, inplace=True)

In [14]:
####################################################
## Merge the adres dataset onto the zaken dataset ##
####################################################

# Merge the adres dataset onto the zaken dataset.
zakenDataset.data = zakenDataset.data.merge(adresDataset.data, on='adres_id', how='left')

# Perform  Feature Extraction

In [15]:
#################################################
## Perform feature extraction on zaken dataset ##
#################################################

categorical_col_hot_zaken = ['afg_code_beh', 'beh_code', 'eigenaar', 'categorie']
categorical_cols_hot_adres = ['toev', 'pvh_omschr', 'sbw_omschr', 'sbv_omschr']
categorical_cols_hot_bag = ['status_coordinaat_code', 'type_woonobject_omschrijving',
                            'eigendomsverhouding_id', 'financieringswijze_id',
                            'gebruik_id', 'ligging_id', 'reden_opvoer_id',
                            'status_id_nummeraanduiding', 'toegang_id']

zakenPipeline = Pipeline(steps=[
    ('extract', FeatureExtractionTransformer(
        categorical_cols_hot=categorical_col_hot_zaken + categorical_cols_hot_adres + categorical_cols_hot_bag,
        ))
    ])

zakenDataset.data = zakenPipeline.fit_transform(zakenDataset.data)

Now extracting features from column: 'afg_code_beh'.
Done!
Now extracting features from column: 'beh_code'.
Done!
Now extracting features from column: 'eigenaar'.
Done!
Now extracting features from column: 'categorie'.
Done!
Now extracting features from column: 'toev'.
Done!
Now extracting features from column: 'pvh_omschr'.
Done!
Now extracting features from column: 'sbw_omschr'.
Done!
Now extracting features from column: 'sbv_omschr'.
Done!
Now extracting features from column: 'status_coordinaat_code'.
Done!
Now extracting features from column: 'type_woonobject_omschrijving'.
Done!
Now extracting features from column: 'eigendomsverhouding_id'.
Done!
Now extracting features from column: 'financieringswijze_id'.
Done!
Now extracting features from column: 'gebruik_id'.
Done!
Now extracting features from column: 'ligging_id'.
Done!
Now extracting features from column: 'reden_opvoer_id'.
Done!
Now extracting features from column: 'status_id_nummeraanduiding'.
Done!
Now extracting features

# Save Finalized Dataset

In [16]:
# Save.
zakenDataset.version = 'final'
zakenDataset.save()

Saving version 'final' of dataframe 'zaken'.


/usr/local/lib/python3.6/site-packages/pandas/core/generic.py:2377: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block5_values] [items->['afg_code_afs', 'afg_code_beh', 'afs_code', 'afs_oms', 'beh_code', 'beh_oms', 'categorie', 'eigenaar', 'mededelingen', 'zaak_id', 'pvh_omschr', 'sbw_omschr', 'sbv_omschr', 'sttnaam', 'hsltr', 'toev', 'sdl_naam', 'postcode_x', 'id_nummeraanduiding', 'landelijk_id_nummeraanduiding', 'huisletter_nummeraanduiding', 'huisnummer_toevoeging_nummeraanduiding', 'type', 'adres_nummer', 'bron_id_nummeraanduiding', 'ligplaats_id', 'openbare_ruimte_id', 'standplaats_id', 'status_id_nummeraanduiding', 'verblijfsobject_id', '_geom', 'bron_id_ligplaats', 'status_id_ligplaats', 'geometrie_standplaats', 'bron_id_standplaats', 'status_id_standplaats', 'status_coordinaat_code', 'type_woonobject_omschrijving', 'geometrie_verblijfsobject', '_huisletter_verblij

In [21]:
adresDataset = AdresDataset()
zakenDataset = ZakenDataset()
stadiaDataset = StadiaDataset()

AttributeError: 'ZakenDataset' object has no attribute 'head'

In [23]:
adres = adresDataset.data
zaken = zakenDataset.data
stadia = stadiaDataset.data

In [25]:
zaken_adres = pd.merge(zaken, adres, how='left', on='adres_id')

In [26]:
zaken_adres.head()

,adres_id,afg_code_afs,afg_code_beh,afs_code,afs_oms,bedrag_huur,begindatum,beh_code,beh_oms,categorie,...,percentage_gezinsverhouding_3_y,gezinsverhouding_4_y,percentage_gezinsverhouding_4_y,gezinsverhouding_5_y,percentage_gezinsverhouding_5_y,gezinsverhouding_6_y,percentage_gezinsverhouding_6_y,gezinsverhouding_7_y,percentage_gezinsverhouding_7_y,aantal_hotline_meldingen_y
0,249960.0,vtbd,vtbd,pgwg,zl geen woonfraude,447.000000,2015-07-27,zkl_hos,zkl illegaal hotel/shortstay,illegale hotels,...,0.0,0,0.0,0,0.0,1,0.25,0,0.0,0
1,407390.0,vtbd,vtbd,pwibg,zl woning is beschikbaar gekomen,259.000000,2014-03-26,zkl_dr_drg,zkl doorzon drugs,drugs,...,0.0,0,0.0,0,0.0,3,1.00,0,0.0,0
2,1937100.0,vtbd,vtbd,pwibg,zl woning is beschikbaar gekomen,467.113891,2013-12-23,zkl_dr_drg,zkl doorzon drugs,drugs,...,0.0,0,0.0,0,0.0,3,1.00,0,0.0,0
3,259342.0,vtbd,vtbd,pwibg,zl woning is beschikbaar gekomen,467.113891,2014-06-18,zkl_dr_drg,zkl doorzon drugs,drugs,...,0.0,0,0.0,0,0.0,0,0.00,0,0.0,0
4,97195.0,vtbd,vtbd,pwibg,zl woning is beschikbaar gekomen,307.000000,2014-11-28,zkl_dr_drg,zkl doorzon drugs,drugs,...,0.0,0,0.0,0,0.0,1,1.00,0,0.0,0


In [30]:
zaken_totaal = pd.merge(zaken_adres, stadia, how='left', on=['adres_id','wvs_nr'])

In [31]:
zaken_totaal.head()

,adres_id,afg_code_afs_x,afg_code_beh,afs_code_x,afs_oms_x,bedrag_huur,begindatum_x,beh_code,beh_oms,categorie,...,date_created,date_modified,wzs_id_y,stadia_id,wzs_update_datumtijd_y,zaak_id_y,stadium_id,label,mask,before_2013
0,249960.0,vtbd,vtbd,pgwg,zl geen woonfraude,447.000000,2015-07-27,zkl_hos,zkl illegaal hotel/shortstay,illegale hotels,...,NaT,NaT,332304164.0,249960_2_2,2019-08-28 13:09:32.065550,249960_2,249960_2_2,misschien,False,False
1,249960.0,vtbd,vtbd,pgwg,zl geen woonfraude,447.000000,2015-07-27,zkl_hos,zkl illegaal hotel/shortstay,illegale hotels,...,NaT,NaT,332357839.0,249960_2_1,2019-08-28 13:09:32.065550,249960_2,249960_2_1,misschien,False,False
2,407390.0,vtbd,vtbd,pwibg,zl woning is beschikbaar gekomen,259.000000,2014-03-26,zkl_dr_drg,zkl doorzon drugs,drugs,...,NaT,NaT,332369818.0,407390_3_1,2019-08-28 13:09:32.065550,407390_3,407390_3_1,misschien,False,False
3,1937100.0,vtbd,vtbd,pwibg,zl woning is beschikbaar gekomen,467.113891,2013-12-23,zkl_dr_drg,zkl doorzon drugs,drugs,...,NaT,NaT,332276522.0,1937100_1_1,2019-08-28 13:09:32.065550,1937100_1,1937100_1_1,misschien,False,False
4,1937100.0,vtbd,vtbd,pwibg,zl woning is beschikbaar gekomen,467.113891,2013-12-23,zkl_dr_drg,zkl doorzon drugs,drugs,...,NaT,NaT,332356952.0,1937100_1_2,2019-08-28 13:09:32.065550,1937100_1,1937100_1_2,misschien,False,False


In [35]:
for col in zaken_totaal.columns: 
    print(col) 

adres_id
afg_code_afs_x
afg_code_beh
afs_code_x
afs_oms_x
bedrag_huur
begindatum_x
beh_code
beh_oms
categorie
eigenaar
einddatum_x
kamer_aantal
mededelingen
nuttig_woonoppervlak
vloeroppervlak_totaal
wvs_nr
wzs_id_x
wzs_update_datumtijd_x
zaak_id_x
woonfraude
pvh_omschr_x
sbw_omschr_x
sbv_omschr_x
wzs_lon_x
wzs_lat_x
sttnaam_x
hsnr_x
hsltr_x
toev_x
sdl_naam_x
inwnrs_x
leegstand_x
ads_id_x
wng_id_x
postcode_x_x
id_nummeraanduiding_x
landelijk_id_nummeraanduiding_x
huisnummer_nummeraanduiding_x
huisletter_nummeraanduiding_x
huisnummer_toevoeging_nummeraanduiding_x
type_x
adres_nummer_x
hoofdadres_x
bron_id_nummeraanduiding_x
ligplaats_id_x
openbare_ruimte_id_x
standplaats_id_x
status_id_nummeraanduiding_x
verblijfsobject_id_x
_geom_x
bron_id_ligplaats_x
status_id_ligplaats_x
geometrie_standplaats_x
bron_id_standplaats_x
status_id_standplaats_x
oppervlakte_x
bouwlaag_toegang_x
status_coordinaat_code_x
bouwlagen_x
type_woonobject_omschrijving_x
aantal_kamers_x
geometrie_verblijfsobject_x
_

In [74]:
bd_lijst = []
for value in zaken_totaal['sta_oms']:
    if 'onderzoek' in str(value).lower() or 'avond' in str(value).lower() or 'controle' in str(value).lower():
        bd_lijst.append(value)
        
bd_set = set(bd_lijst)
print (bd_set)


{'geen onderzoek', '2de controle', '2de hercontrole', 'kwo onderzoek', 'hercontrole boete', 'avondronde', 'weekend buitendienstonderzoek', 'hercontrole', 'avondronde hercontrole', 'dossieronderzoek', 'buitendienstonderzoek', 'onderzoek advertentie', 'onderzoek buitendienst', 'eindcontrole', 'aanvullend onderzoek in behandeling od', 'controle dwangsom', '3de controle', 'bureauonderzoek', '3de hercontrole'}


In [75]:
#lijst van alle Buitendienst stadia
bd_lijst = ['2de controle','3de controle','onderzoek buitendienst','avondronde','weekend buitendienstonderzoek','buitendienstonderzoek']


In [44]:
indicatie_buitendienst = []

#create and indication buitendienst
for value in zaken_totaal['sta_oms']:
    if value in bd_lijst:
        indicatie_buitendienst.append(1)
    else:
        indicatie_buitendienst.append(0)

In [45]:
sum(indicatie_buitendienst)

18992

In [46]:
# add indicatie buitendienst to zaken_totaal
zaken_totaal['indicatie_buitendienst'] = indicatie_buitendienst

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
11       0
12       0
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       1
23       0
24       0
25       0
26       0
27       0
28       0
29       0
        ..
89248    1
89249    1
89250    0
89251    0
89252    0
89253    0
89254    0
89255    1
89256    0
89257    1
89258    0
89259    0
89260    0
89261    0
89262    0
89263    0
89264    0
89265    0
89266    0
89267    0
89268    1
89269    0
89270    0
89271    0
89272    1
89273    1
89274    0
89275    0
89276    0
89277    0
Name: indicatie_buitendienst, Length: 89278, dtype: int64

In [63]:
def BD_belopen(df):

    if (df['indicatie_buitendienst'] == 1 and str(df['einddatum_x']) != ''  and str(df['einddatum_y']) != ''):
        return 1
    else:
        return 0


zaken_totaal['indicatie_buitendienst_belopen'] = zaken_totaal.apply(BD_belopen, axis = 1)

In [64]:
BD_opgevoerd = sum(zaken_totaal['indicatie_buitendienst'])
BD_belopen = sum(zaken_totaal['indicatie_buitendienst_belopen'])

print(BD_opgevoerd)
print(BD_belopen)

18992
18992


In [71]:
zaken_totaal['categorie'].head()

BD_per_project = zaken_totaal.groupby(['categorie'])['indicatie_buitendienst_belopen'].sum()

BD_per_project


categorie
buurtprojecten                               260
drugs                                          3
illegale hotels                             8821
illegale onderhuur                           275
in gebruik zonder huisvestingsvergunning      29
inschrijvingsfraude                          109
laa                                         2122
leegstand                                   3182
missing                                      890
onrechtmatige bewoning                      1152
prostitutie                                   26
uitkeringsfraude                               0
woningdelen                                 2123
Name: indicatie_buitendienst_belopen, dtype: int64

In [73]:
bd_lijst = []
for value in zaken_totaal['sta_oms']:
    if 'last' in str(value).lower() or 'boete' in str(value).lower() or 'sluit' in str(value).lower() or 'meldplic' in str(value).lower():
        bd_lijst.append(value)
        
bd_set = set(bd_lijst)
print (bd_set)

{'opleggen boete', 'last onder dwangsom', 'opleggen boete doorverhuurder', 'last tot sluiting', 'meldplicht beschikking boete', 'voornemen boete', 'last onder bestuursdwang', 'vooraankondiging last tot sluiting', 'opheffen last', 'bew(huid) last onder dwangsom', 'vav geluidsoverlast', 'meldplicht hanzlt', 'vooraankondiging bestuurlijke boete', 'meldplicht voornemen boete', 'hercontrole boete', 'voornemen boete doorverhuurder', 'eig. last onder dwangsom', 'bestuurlijke boete zaak', 'last maken', 'boete maken', 'voornemen boete eigenaar', 'zl belastingdienst toeslagen', 'sluiting (ivm brandveiligheid)', 'opleggen boete eigenaar', 'belasting debiteuren', 'intrekken besluit'}


In [77]:
hb_lijst = ['opleggen boete','last onder dwangsom','opleggen boete doorverhuurder','meldplicht beschikking boete','last onder bestuursdwang','bew(huid) last onder dwangsom','eig. last onder dwangsom','bestuurlijke boete zaak','sluiting (ivm brandveiligheid)','opleggen boete eigenaar']

hb_lijst

['opleggen boete',
 'last onder dwangsom',
 'opleggen boete doorverhuurder',
 'meldplicht beschikking boete',
 'last onder bestuursdwang',
 'bew(huid) last onder dwangsom',
 'eig. last onder dwangsom',
 'bestuurlijke boete zaak',
 'sluiting (ivm brandveiligheid)',
 'opleggen boete eigenaar']

In [78]:
indicatie_handhaving = []

#create and indication buitendienst
for value in zaken_totaal['sta_oms']:
    if value in hb_lijst:
        indicatie_handhaving.append(1)
    else:
        indicatie_handhaving.append(0)

In [79]:
# add indicatie buitendienst to zaken_totaal
zaken_totaal['indicatie_handhaving'] = indicatie_handhaving

In [80]:
zaken_totaal['categorie'].head()

HB_per_project = zaken_totaal.groupby(['categorie'])['indicatie_handhaving'].sum()

HB_per_project

categorie
buurtprojecten                                13
drugs                                          3
illegale hotels                             1709
illegale onderhuur                             2
in gebruik zonder huisvestingsvergunning       4
inschrijvingsfraude                            4
laa                                            0
leegstand                                     19
missing                                      113
onrechtmatige bewoning                        90
prostitutie                                    3
uitkeringsfraude                               0
woningdelen                                  533
Name: indicatie_handhaving, dtype: int64

In [82]:
HB_per_BD = HB_per_project/BD_per_project

In [83]:
HB_per_BD

categorie
buurtprojecten                              0.050000
drugs                                       1.000000
illegale hotels                             0.193742
illegale onderhuur                          0.007273
in gebruik zonder huisvestingsvergunning    0.137931
inschrijvingsfraude                         0.036697
laa                                         0.000000
leegstand                                   0.005971
missing                                     0.126966
onrechtmatige bewoning                      0.078125
prostitutie                                 0.115385
uitkeringsfraude                                 NaN
woningdelen                                 0.251060
dtype: float64

In [84]:
HB_per_project

categorie
buurtprojecten                                13
drugs                                          3
illegale hotels                             1709
illegale onderhuur                             2
in gebruik zonder huisvestingsvergunning       4
inschrijvingsfraude                            4
laa                                            0
leegstand                                     19
missing                                      113
onrechtmatige bewoning                        90
prostitutie                                    3
uitkeringsfraude                               0
woningdelen                                  533
Name: indicatie_handhaving, dtype: int64

In [85]:
BD_per_project

categorie
buurtprojecten                               260
drugs                                          3
illegale hotels                             8821
illegale onderhuur                           275
in gebruik zonder huisvestingsvergunning      29
inschrijvingsfraude                          109
laa                                         2122
leegstand                                   3182
missing                                      890
onrechtmatige bewoning                      1152
prostitutie                                   26
uitkeringsfraude                               0
woningdelen                                 2123
Name: indicatie_buitendienst_belopen, dtype: int64